In [1]:
import numpy as np

In [2]:
from auctions import GeneralizedFirstPriceAuction
from interactions import Customer

In [3]:
n_advertisers = 5
n_slots = 3

ctrs = np.ones(n_advertisers)
lambdas = np.ones(n_slots)

bids = [0.5, 0.2, 0.7, 0.1, 0.9]

In [4]:
auction = GeneralizedFirstPriceAuction(ctrs, lambdas)

winners, payments_per_click = auction.round(bids)

print("Winners: ", winners)
print("Payments per click: ", payments_per_click)

Winners:  [4 2 0]
Payments per click:  [0.9, 0.7, 0.5]


In [5]:
auction.get_click_through_rates()

array([1., 1., 1., 1., 1.])

In [6]:
class Company:
    def __init__(self, pricing_agent, bidding_agent, budget, valuation, product_cost):
        self.pricing_agent = pricing_agent
        self.bidding_agent = bidding_agent
        self.budget = budget
        self.valuation = valuation
        self.product_cost = product_cost
    
    def set_price(self):
        return self.pricing_agent.set_price()

    def bid(self):
        return self.bidding_agent.bid()
    
    def update_pricing_strategy(self, purchase_reward: int):
        self.pricing_agent.update(purchase_reward)

    def update_bidding_strategy(self, auction_results):
        # Compute utility and cost
        f_t, c_t = self._compute_utility_and_cost(auction_results)
        m_t = auction_results['m_t']

        # Update bidding strategy
        self.bidding_agent.update(f_t, c_t, m_t)
    
    def _compute_utility_and_cost(self, auction_results):
        win = auction_results['company_win']
        m_t = auction_results['m_t']

        # Compute utility
        f_t = (self.valuation - m_t) * win

        # Compute cost
        c_t = m_t * win

        return f_t, c_t

In [7]:
class Publisher:
    # The company index is always 0, as the company bid is the first one
    COMPANY_INDEX = 0

    def __init__(self, auction, competitors):
        self.auction = auction
        self.competitors = competitors
    
    def round(self, bid):
        # Get the competitor bids and the maximum among them
        competitor_bids, m_t = self.competitors.get_bids()

        # Append the competitor bids to the company's bid
        bids = np.append(bid, competitor_bids)

        # Run the auction
        winners, payments_per_click = self.auction.round(bids)
        
        # Check if the company won the auction, and if so, the slot in which the ad was shown
        company_win = self.COMPANY_INDEX in winners
        company_slot = np.where(winners == self.COMPANY_INDEX)[0][0] if company_win else -1

        # Get the company's click through rate
        ctrs = self.auction.get_click_through_rates()
        company_ctr = ctrs[self.COMPANY_INDEX]

        # Simulate the click outcome (True if the user clicked on the ad)
        click_outcome = np.random.rand() < company_ctr

        auction_results = {
            'company_win': company_win,
            'company_slot': company_slot,
            'm_t': m_t
        }

        return auction_results, click_outcome

In [8]:
class Competitors:
    # TODO: implement
    # Define behavior, distributions to sample bids from, etc...
    # Best to define a Competitors abstract class, and then implement different strategies
    # Such as StochasticCompetitors, AdversarialCompetitors, etc...
    ...

In [9]:
class Competitors:
    def __init__(self, *args, **kwargs):
        pass

    def get_bids(self):
        pass

In [10]:
class StochasticCompetitors(Competitors):
    def __init__(self, n_competitors, distribution):
        self.n_competitors = n_competitors
        self.distribution = distribution

    def get_bids(self):
        # Sample bids from the distribution
        bids = self.distribution(self.n_competitors)

        # Get the maximum bid
        m_t = np.max(bids)

        return bids, m_t

In [15]:
np.random.seed(0)

distribution = lambda n_competitors: np.random.uniform(0, 1, n_competitors)

n_competitors = 5
competitors = StochasticCompetitors(n_competitors, distribution)

competitors.get_bids()

(array([0.5488135 , 0.71518937, 0.60276338, 0.54488318, 0.4236548 ]),
 0.7151893663724195)

In [35]:
competitors.get_bids()

(array([0.67781654, 0.27000797, 0.73519402, 0.96218855, 0.24875314]),
 0.9621885451174382)

In [ ]:
class Interaction:
    def __init__(self, company, publisher, customer):
        self.company = company
        self.publisher = publisher
        self.customer = customer
    
    def day(self, n_users):
        # Simulates a day, made up of multiple auctions

        # Company sets a price
        price = self.company.set_price()

        # Company faces a series of auctions
        for user in range(n_users):
            # Company bids
            bid = self.company.bid()

            # Publisher runs the auction and simulates the click outcome
            auction_results, click_outcome = self.publisher.round(bid)

            # Update the company's bidding strategy
            self.company.update_bidding_strategy(auction_results)

            # Update the company's pricing strategy, but only if the user clicked on the ad
            if click_outcome:
                purchase_reward = self.customer.check_purchase(price) # 1 if the user purchased, 0 otherwise
                self.company.update_pricing_strategy(purchase_reward)

    # TODO: continue if there's more to do
    ...

In [ ]:
# Generic structure of the simulation
n_days = ...
n_users = ...

company = Company(...)
publisher = Publisher(...)
competitors = Competitors(...)
customer = Customer(...)

interaction = Interaction(company, publisher, competitors, customer)

for day in range(n_days):
    interaction.day(n_users)

    # TODO: continue
    ...